In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEVELOPMENT_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    %pip install circuitsvis

    # PySvelte is an unmaintained visualization library, use it as a backup if circuitsvis isn't working
    # # Install another version of node that makes PySvelte work way faster
    # !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    # %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Jupyter notebook - intended for development only!


In [2]:
%pip install git+https://github.com/neelnanda-io/TransformerLens.git
%pip install circuitsvis

  Cloning https://github.com/neelnanda-io/TransformerLens.git to c:\users\woog\appdata\local\temp\pip-req-build-um17ki97
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 3cd943628b5c415585c8ef100f65989f6adc7f75
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git 'C:\Users\Woog\AppData\Local\Temp\pip-req-build-um17ki97'


Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import numpy as np
from transformer_lens import HookedTransformer, HookedTransformerConfig

In [22]:
%conda remove pytorch torchvision torchaudio cudatoolkit
%pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

Solving environment: ...working... failed

Note: you may need to restart the kernel to use updated packages.



PackagesNotFoundError: The following packages are missing from the target environment:
  - cudatoolkit


UsageError: Line magic function `%pip3` not found.


In [2]:
import random

In [3]:
def generate_digit_dataset(n_examples, n_digits, reversed_output=False, n_equals=0):
    # Generate a dataset of equations that involve an addition between two numbers with n_digits
    assert n_examples <= 10 ** (n_digits * 2)
    dataset = set()
    while len(dataset) != n_examples:
        # Sample random numbers
        if balanced_train:
            i = random.randint(1,n_digits)
            a = np.random.randint(0, 10**i)
            b = np.random.randint(0, 10**i)
        else:
          a = np.random.randint(0, 10**n_digits)
          b = np.random.randint(0, 10**n_digits)
        c = str(a+b).zfill(n_digits+1)
        if reversed_output:
            c = c[::-1] # Just reverse the order of the characters in the answer
        dataset.add(f"{str(a).zfill(n_digits)}{str(b).zfill(n_digits)}{n_equals * '='}{c}")
    return list(dataset)

def tokenize_strings(strings):
    vocab = "0123456789="
    vocab2idx = {v: i for i, v in enumerate(vocab)}
    # Tokenize a list of strings
    tokens = np.zeros((len(strings), len(strings[0])), dtype=np.int64)
    for i, string in enumerate(strings):
        idxs = [vocab2idx[char] for char in list(string)]
        tokens[i] = np.array(idxs)
    return tokens

class TokenizedDataset(torch.utils.data.Dataset):

    def __init__(self, tokens):
        # Tokens should be of shape N_examples x N_chars x D_tokens
        self.tokens = torch.tensor(tokens).long()

    def __getitem__(self, index):
        # Should return an example of N_chars x D_tokens
        return self.tokens[index]

    def __len__(self):
        return self.tokens.shape[0]

In [98]:
# Generate a simple dataset with the specified params
n_digits = 3
# n_train_set = 100_000
# n_test_set = 10_000
n_train_set = 500
n_test_set = 100
# reversed_output = False
n_equals = 0
reversed_output = True
balanced_train = True

strings = generate_digit_dataset(n_train_set + n_test_set, n_digits, reversed_output, n_equals)
#strings_straight = generate_digit_dataset(n_train_set + n_test_set, n_digits, False, n_equals)
tokens = tokenize_strings(strings)
#tokens_straight = tokenize_strings(strings_straight)
train_set = TokenizedDataset(tokens[:n_train_set])
test_set = TokenizedDataset(tokens[n_train_set:])
#train_set_straight = TokenizedDataset(tokens_straight[:n_train_set])
#test_set_straight = TokenizedDataset(tokens_straight[n_train_set:])

# Create Dataloaders
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False, drop_last=True)
#train_loader_straight = torch.utils.data.DataLoader(train_set_straight, batch_size=64, shuffle=True, drop_last=True)
#test_loader_straight = torch.utils.data.DataLoader(test_set_straight, batch_size=64, shuffle=False, drop_last=True)

In [124]:
n_layers = 1
n_heads = 2
dmodel = 20

In [125]:
n_epochs = 500

In [126]:
def construct_model(n_layers, n_heads, attn_only=False, disable_b=False):
    # Create a HookedTransformer
    cfg = HookedTransformerConfig(
        n_layers=n_layers,
        n_heads=n_heads,
        d_model=16, # previously 128
        d_head=8, # previously 32
        d_mlp=9 if not attn_only else None, # previously 512
        act_fn="gelu" if not attn_only else None,
        attn_only=attn_only,
        normalization_type=None,
        attention_dir="causal",
        d_vocab=12,
        d_vocab_out=10,
        n_ctx=3*n_digits+1+n_equals, # Create appropriate context size for strings
        init_weights=True,
        device="cuda"
    )
    model = HookedTransformer(cfg)
    if disable_b:
        for name, param in model.named_parameters():
            if "b_" in name:
                param.requires_grad = False
    return model

def loss_fn(logits, labels):
    # Logits is shape B x P x D, labels is B x P
    log_probs = logits.reshape(-1, logits.shape[-1]).log_softmax(dim=-1)
    labels = labels.reshape(-1)
    correct_log_probs = log_probs.gather(dim=-1, index=labels[:, None])[:, 0]
    return -correct_log_probs.mean()

def acc_fn(logits, labels):
    with torch.no_grad():
        # We can't differentiate this
        preds = logits.argmax(dim=-1)
        correct = preds == labels
        return torch.sum(correct).item() / np.prod(correct.shape)


def train_model(model, train_loader, test_loader, n_epochs, verbose=False,):
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, ) #betas=(0.9, 0.98), weight_decay=1.,)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs)

    train_acc = []
    test_acc = []

    for epoch in range(n_epochs):

        acc = []
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            n_batch, n_pos = batch.shape
            cutoff = n_pos - n_digits - 1
            X = torch.cat((batch[:, :cutoff], 10 * torch.ones(n_batch, n_digits + 1, dtype=torch.long)), dim=1).cuda() # Pad with = where the output should be
            labels = batch[:, cutoff:].cuda()
            logits = model(X)[:, cutoff:]
            loss = loss_fn(logits, labels)
            loss.backward()
            optimizer.step()
            acc.append(acc_fn(logits, labels))
        train_acc.append(np.array(acc).mean())


        acc = []
        model.eval()
        with torch.no_grad():
            for batch in test_loader:
                n_batch, n_pos = batch.shape
                cutoff = n_pos - n_digits - 1
                X = torch.cat((batch[:, :cutoff], 10 * torch.ones(n_batch, n_digits + 1, dtype=torch.long)), dim=1).cuda()
                labels = batch[:, cutoff:].cuda()
                logits = model(X)[:, cutoff:]
                acc.append(acc_fn(logits, labels))
        test_acc.append(np.array(acc).mean())

        scheduler.step()
        if verbose:
            print(f"Epoch: {epoch + 1}, Train Acc: {train_acc[-1]*100:.3f}%, Test Acc: {test_acc[-1]*100:.3f}%")

    return train_acc, test_acc

# Reversed performance

In [9]:
torch.cuda.is_available()

True

In [127]:
# Train model, 5 digit reversed, balanced
model = construct_model(n_layers, n_heads, attn_only=True)
train_acc, test_acc = train_model(model, train_loader, test_loader, n_epochs, verbose=True)

Epoch: 1, Train Acc: 5.469%, Test Acc: 6.641%
Epoch: 2, Train Acc: 15.346%, Test Acc: 25.781%
Epoch: 3, Train Acc: 26.004%, Test Acc: 25.781%
Epoch: 4, Train Acc: 30.636%, Test Acc: 36.328%
Epoch: 5, Train Acc: 36.775%, Test Acc: 35.156%
Epoch: 6, Train Acc: 36.663%, Test Acc: 36.328%
Epoch: 7, Train Acc: 36.161%, Test Acc: 37.500%
Epoch: 8, Train Acc: 36.161%, Test Acc: 37.500%
Epoch: 9, Train Acc: 36.886%, Test Acc: 37.500%
Epoch: 10, Train Acc: 36.496%, Test Acc: 37.500%
Epoch: 11, Train Acc: 36.384%, Test Acc: 37.500%
Epoch: 12, Train Acc: 36.998%, Test Acc: 37.500%
Epoch: 13, Train Acc: 35.938%, Test Acc: 37.500%
Epoch: 14, Train Acc: 36.496%, Test Acc: 37.500%
Epoch: 15, Train Acc: 36.775%, Test Acc: 37.500%
Epoch: 16, Train Acc: 36.496%, Test Acc: 37.500%
Epoch: 17, Train Acc: 36.942%, Test Acc: 37.109%
Epoch: 18, Train Acc: 36.607%, Test Acc: 37.500%
Epoch: 19, Train Acc: 37.054%, Test Acc: 37.500%
Epoch: 20, Train Acc: 37.388%, Test Acc: 37.109%
Epoch: 21, Train Acc: 37.388%, 

Epoch: 168, Train Acc: 51.116%, Test Acc: 46.484%
Epoch: 169, Train Acc: 51.283%, Test Acc: 44.922%
Epoch: 170, Train Acc: 52.511%, Test Acc: 45.703%
Epoch: 171, Train Acc: 51.283%, Test Acc: 45.312%
Epoch: 172, Train Acc: 51.953%, Test Acc: 45.312%
Epoch: 173, Train Acc: 51.953%, Test Acc: 46.484%
Epoch: 174, Train Acc: 51.953%, Test Acc: 45.312%
Epoch: 175, Train Acc: 51.339%, Test Acc: 45.312%
Epoch: 176, Train Acc: 52.232%, Test Acc: 45.312%
Epoch: 177, Train Acc: 52.567%, Test Acc: 47.266%
Epoch: 178, Train Acc: 52.623%, Test Acc: 46.875%
Epoch: 179, Train Acc: 52.902%, Test Acc: 47.266%
Epoch: 180, Train Acc: 52.846%, Test Acc: 46.094%
Epoch: 181, Train Acc: 52.400%, Test Acc: 47.656%
Epoch: 182, Train Acc: 52.958%, Test Acc: 47.266%
Epoch: 183, Train Acc: 52.288%, Test Acc: 46.875%
Epoch: 184, Train Acc: 52.288%, Test Acc: 46.875%
Epoch: 185, Train Acc: 51.451%, Test Acc: 48.047%
Epoch: 186, Train Acc: 52.232%, Test Acc: 45.703%
Epoch: 187, Train Acc: 52.846%, Test Acc: 45.703%


Epoch: 333, Train Acc: 66.797%, Test Acc: 56.641%
Epoch: 334, Train Acc: 65.458%, Test Acc: 56.641%
Epoch: 335, Train Acc: 66.071%, Test Acc: 57.031%
Epoch: 336, Train Acc: 66.797%, Test Acc: 56.641%
Epoch: 337, Train Acc: 66.071%, Test Acc: 57.031%
Epoch: 338, Train Acc: 66.071%, Test Acc: 57.812%
Epoch: 339, Train Acc: 67.020%, Test Acc: 57.812%
Epoch: 340, Train Acc: 67.188%, Test Acc: 58.203%
Epoch: 341, Train Acc: 67.188%, Test Acc: 58.984%
Epoch: 342, Train Acc: 66.908%, Test Acc: 58.984%
Epoch: 343, Train Acc: 66.964%, Test Acc: 58.203%
Epoch: 344, Train Acc: 67.243%, Test Acc: 58.594%
Epoch: 345, Train Acc: 67.690%, Test Acc: 58.594%
Epoch: 346, Train Acc: 68.192%, Test Acc: 58.594%
Epoch: 347, Train Acc: 68.248%, Test Acc: 59.375%
Epoch: 348, Train Acc: 67.578%, Test Acc: 59.375%
Epoch: 349, Train Acc: 67.467%, Test Acc: 59.375%
Epoch: 350, Train Acc: 68.136%, Test Acc: 60.156%
Epoch: 351, Train Acc: 68.248%, Test Acc: 60.156%
Epoch: 352, Train Acc: 67.578%, Test Acc: 60.547%


Epoch: 498, Train Acc: 71.484%, Test Acc: 62.500%
Epoch: 499, Train Acc: 71.819%, Test Acc: 62.500%
Epoch: 500, Train Acc: 71.819%, Test Acc: 62.500%


In [129]:
# Train more
train_acc, test_acc = train_model(model, train_loader, test_loader, 1000, verbose=True)

Epoch: 1, Train Acc: 81.808%, Test Acc: 75.391%
Epoch: 2, Train Acc: 82.254%, Test Acc: 75.391%
Epoch: 3, Train Acc: 83.482%, Test Acc: 77.734%
Epoch: 4, Train Acc: 83.036%, Test Acc: 76.953%
Epoch: 5, Train Acc: 83.817%, Test Acc: 75.391%
Epoch: 6, Train Acc: 83.929%, Test Acc: 77.344%
Epoch: 7, Train Acc: 83.203%, Test Acc: 76.953%
Epoch: 8, Train Acc: 83.203%, Test Acc: 76.953%
Epoch: 9, Train Acc: 83.259%, Test Acc: 75.000%
Epoch: 10, Train Acc: 83.371%, Test Acc: 77.344%
Epoch: 11, Train Acc: 83.873%, Test Acc: 77.344%
Epoch: 12, Train Acc: 83.929%, Test Acc: 76.562%
Epoch: 13, Train Acc: 84.040%, Test Acc: 76.562%
Epoch: 14, Train Acc: 83.147%, Test Acc: 77.344%
Epoch: 15, Train Acc: 83.984%, Test Acc: 77.734%
Epoch: 16, Train Acc: 83.426%, Test Acc: 76.953%
Epoch: 17, Train Acc: 83.929%, Test Acc: 77.344%
Epoch: 18, Train Acc: 82.924%, Test Acc: 76.953%
Epoch: 19, Train Acc: 84.040%, Test Acc: 76.953%
Epoch: 20, Train Acc: 83.482%, Test Acc: 76.953%
Epoch: 21, Train Acc: 83.594%

Epoch: 168, Train Acc: 83.650%, Test Acc: 75.781%
Epoch: 169, Train Acc: 84.542%, Test Acc: 76.562%
Epoch: 170, Train Acc: 83.929%, Test Acc: 77.344%
Epoch: 171, Train Acc: 83.705%, Test Acc: 77.734%
Epoch: 172, Train Acc: 83.873%, Test Acc: 77.734%
Epoch: 173, Train Acc: 83.817%, Test Acc: 76.953%
Epoch: 174, Train Acc: 84.096%, Test Acc: 76.172%
Epoch: 175, Train Acc: 83.817%, Test Acc: 77.344%
Epoch: 176, Train Acc: 84.263%, Test Acc: 76.953%
Epoch: 177, Train Acc: 84.542%, Test Acc: 76.953%
Epoch: 178, Train Acc: 83.594%, Test Acc: 77.344%
Epoch: 179, Train Acc: 84.208%, Test Acc: 77.734%
Epoch: 180, Train Acc: 84.487%, Test Acc: 76.172%
Epoch: 181, Train Acc: 83.203%, Test Acc: 76.562%
Epoch: 182, Train Acc: 83.929%, Test Acc: 77.344%
Epoch: 183, Train Acc: 83.705%, Test Acc: 77.344%
Epoch: 184, Train Acc: 83.426%, Test Acc: 76.172%
Epoch: 185, Train Acc: 84.263%, Test Acc: 77.734%
Epoch: 186, Train Acc: 84.319%, Test Acc: 77.344%
Epoch: 187, Train Acc: 83.092%, Test Acc: 76.953%


Epoch: 334, Train Acc: 84.263%, Test Acc: 76.953%
Epoch: 335, Train Acc: 84.375%, Test Acc: 77.734%
Epoch: 336, Train Acc: 84.877%, Test Acc: 77.344%
Epoch: 337, Train Acc: 84.933%, Test Acc: 76.953%
Epoch: 338, Train Acc: 84.598%, Test Acc: 76.953%
Epoch: 339, Train Acc: 84.766%, Test Acc: 76.562%
Epoch: 340, Train Acc: 85.379%, Test Acc: 77.734%
Epoch: 341, Train Acc: 84.487%, Test Acc: 77.734%
Epoch: 342, Train Acc: 84.208%, Test Acc: 76.953%
Epoch: 343, Train Acc: 84.877%, Test Acc: 77.344%
Epoch: 344, Train Acc: 84.933%, Test Acc: 77.734%
Epoch: 345, Train Acc: 84.598%, Test Acc: 76.953%
Epoch: 346, Train Acc: 84.542%, Test Acc: 76.953%
Epoch: 347, Train Acc: 85.045%, Test Acc: 76.562%
Epoch: 348, Train Acc: 84.263%, Test Acc: 77.344%
Epoch: 349, Train Acc: 85.045%, Test Acc: 76.953%
Epoch: 350, Train Acc: 84.542%, Test Acc: 76.562%
Epoch: 351, Train Acc: 84.933%, Test Acc: 77.344%
Epoch: 352, Train Acc: 85.045%, Test Acc: 77.344%
Epoch: 353, Train Acc: 84.877%, Test Acc: 77.734%


Epoch: 501, Train Acc: 84.877%, Test Acc: 77.344%
Epoch: 502, Train Acc: 84.710%, Test Acc: 76.562%
Epoch: 503, Train Acc: 84.877%, Test Acc: 77.344%
Epoch: 504, Train Acc: 84.933%, Test Acc: 77.734%
Epoch: 505, Train Acc: 84.598%, Test Acc: 76.562%
Epoch: 506, Train Acc: 84.431%, Test Acc: 76.953%
Epoch: 507, Train Acc: 85.045%, Test Acc: 76.562%
Epoch: 508, Train Acc: 84.654%, Test Acc: 76.953%
Epoch: 509, Train Acc: 84.933%, Test Acc: 77.734%
Epoch: 510, Train Acc: 84.933%, Test Acc: 77.344%
Epoch: 511, Train Acc: 84.933%, Test Acc: 77.734%
Epoch: 512, Train Acc: 84.487%, Test Acc: 78.125%
Epoch: 513, Train Acc: 84.989%, Test Acc: 76.953%
Epoch: 514, Train Acc: 84.487%, Test Acc: 76.172%
Epoch: 515, Train Acc: 84.487%, Test Acc: 76.953%
Epoch: 516, Train Acc: 84.821%, Test Acc: 76.562%
Epoch: 517, Train Acc: 84.487%, Test Acc: 77.344%
Epoch: 518, Train Acc: 85.045%, Test Acc: 76.562%
Epoch: 519, Train Acc: 85.268%, Test Acc: 77.344%
Epoch: 520, Train Acc: 84.431%, Test Acc: 78.125%


Epoch: 668, Train Acc: 84.375%, Test Acc: 77.344%
Epoch: 669, Train Acc: 84.933%, Test Acc: 77.734%
Epoch: 670, Train Acc: 84.821%, Test Acc: 76.953%
Epoch: 671, Train Acc: 85.045%, Test Acc: 77.734%
Epoch: 672, Train Acc: 84.598%, Test Acc: 77.344%
Epoch: 673, Train Acc: 84.319%, Test Acc: 76.562%
Epoch: 674, Train Acc: 84.598%, Test Acc: 77.344%
Epoch: 675, Train Acc: 84.598%, Test Acc: 76.953%
Epoch: 676, Train Acc: 84.375%, Test Acc: 77.734%
Epoch: 677, Train Acc: 84.877%, Test Acc: 77.344%
Epoch: 678, Train Acc: 84.877%, Test Acc: 76.953%
Epoch: 679, Train Acc: 84.487%, Test Acc: 76.953%
Epoch: 680, Train Acc: 84.654%, Test Acc: 77.344%
Epoch: 681, Train Acc: 85.100%, Test Acc: 77.344%
Epoch: 682, Train Acc: 84.877%, Test Acc: 76.953%
Epoch: 683, Train Acc: 84.821%, Test Acc: 76.953%
Epoch: 684, Train Acc: 85.100%, Test Acc: 77.734%
Epoch: 685, Train Acc: 85.156%, Test Acc: 77.344%
Epoch: 686, Train Acc: 84.766%, Test Acc: 76.953%
Epoch: 687, Train Acc: 84.375%, Test Acc: 76.562%


Epoch: 836, Train Acc: 84.487%, Test Acc: 77.734%
Epoch: 837, Train Acc: 84.598%, Test Acc: 77.734%
Epoch: 838, Train Acc: 85.156%, Test Acc: 77.344%
Epoch: 839, Train Acc: 84.821%, Test Acc: 76.953%
Epoch: 840, Train Acc: 84.710%, Test Acc: 77.344%
Epoch: 841, Train Acc: 84.542%, Test Acc: 76.953%
Epoch: 842, Train Acc: 85.100%, Test Acc: 77.344%
Epoch: 843, Train Acc: 84.710%, Test Acc: 77.344%
Epoch: 844, Train Acc: 85.156%, Test Acc: 77.344%
Epoch: 845, Train Acc: 84.654%, Test Acc: 77.344%
Epoch: 846, Train Acc: 85.045%, Test Acc: 77.344%
Epoch: 847, Train Acc: 84.710%, Test Acc: 77.734%
Epoch: 848, Train Acc: 85.045%, Test Acc: 77.734%
Epoch: 849, Train Acc: 84.989%, Test Acc: 77.344%
Epoch: 850, Train Acc: 85.491%, Test Acc: 77.734%
Epoch: 851, Train Acc: 85.156%, Test Acc: 77.344%
Epoch: 852, Train Acc: 84.821%, Test Acc: 77.344%
Epoch: 853, Train Acc: 84.933%, Test Acc: 77.344%
Epoch: 854, Train Acc: 85.156%, Test Acc: 77.344%
Epoch: 855, Train Acc: 85.045%, Test Acc: 77.344%


# Straight performance

# Testing

In [119]:
# Test adding two numbers
def add_sorta_tokenize(a, b):
    
    assert a < (10**n_digits) and b < (10**n_digits)
    input_str = f"{str(a).zfill(n_digits)}{str(b).zfill(n_digits)}{(n_equals+n_digits+1) * '='}"
    print(input_str)
    tokens = tokenize_strings([input_str,])
    return tokens
def add_two_nums(a, b):
    assert a < (10**n_digits) and b < (10**n_digits)
    input_str = f"{str(a).zfill(n_digits)}{str(b).zfill(n_digits)}{(n_equals+n_digits+1) * '='}"
    print(input_str)
    tokens = tokenize_strings([input_str,])
    tokens = torch.tensor(tokens).long().cuda()
    output = model(tokens).argmax(2)
    return output[0, -(n_digits+1):]

add_two_nums(458,123)

458123====


tensor([1, 7, 5, 0], device='cuda:0')

In [122]:
input_str = f"{str(2).zfill(n_digits)}{str(5).zfill(n_digits)}{(n_equals+n_digits+1) * '='}"
tokens = tokenize_strings([input_str,])

_,size = tokens.shape
string_tokens = []
for i in range(size):
  string_tokens.append(str(tokens[0][i]))
print(string_tokens)
tokens = torch.tensor(tokens).long().cuda()
output = model(tokens).argmax(2)

print("Here is the output",output[0, -(n_digits+1):])

logits, cache = model.run_with_cache(tokens)

['0', '0', '2', '0', '0', '5', '10', '10', '10', '10']
Here is the output tensor([7, 0, 0, 0], device='cuda:0')


In [123]:
import circuitsvis as cv
from IPython.display import display

# Assuming you have defined the 'tokens' and 'cache' variables

html = cv.attention.attention_patterns(
    tokens = string_tokens,
    attention=cache["pattern", 0][0]
)

display(html)

Results for 1 layer, 4 heads, 4 digits, 500 train, 300 epochs

results not super replicable. d=20 got 30% sometimes 45% others

there appears to be a phase change around ~36-40%?

d=28 got worse results than 24 for 5 runs until the last one got over a huge learning bump and converged to

| d_model | peak (train%/test%) | min (train%/test%) | reliability |
|-|-|-|-|
|d = 16| 36.8, 28.6 |  |  |
|d = 20| 44.7, 36.2 |  |  |
|d = 24| 82.1, 75.6 |  |  |
|d = 28| 94.6, 91.6 |  |  |
|d = 32|  |  |  |
|d = 36|  |  |  |
|d = 40|  |  |  |
|d = 44|  |  |  |
|d = 48|  |  |  |

In [87]:
torch.save(model.state_dict(), "model-3digit-attn-only.pt")

In [50]:
for name, param in model.named_parameters():
    if name.startswith("blocks.0."):
        print(name, param.shape)

blocks.0.attn.W_Q torch.Size([2, 16, 8])
blocks.0.attn.W_K torch.Size([2, 16, 8])
blocks.0.attn.W_V torch.Size([2, 16, 8])
blocks.0.attn.W_O torch.Size([2, 8, 16])
blocks.0.attn.b_Q torch.Size([2, 8])
blocks.0.attn.b_K torch.Size([2, 8])
blocks.0.attn.b_V torch.Size([2, 8])
blocks.0.attn.b_O torch.Size([16])
blocks.0.mlp.W_in torch.Size([16, 64])
blocks.0.mlp.b_in torch.Size([64])
blocks.0.mlp.W_out torch.Size([64, 16])
blocks.0.mlp.b_out torch.Size([16])


In [52]:
input = f"145+285="
tok = model.to_tokens(input)
print(tok)
#loss = model("145+285=", return_type="loss")
#print("Model loss:", loss)

AssertionError: Cannot use to_tokens without a tokenizer